In [1]:
import pandas as pd
import numpy as np
import traffic
from traffic.core import Traffic
from traffic.data import opensky
from traffic.data import airports
from traffic.data import aircraft
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm 
import importlib
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

## Trajs Preprocessing and Classification (Takeoff & Pushback)

In [33]:
# Select time period (e.g. month) to be downloaded.

airport_str = 'LSZH'

# May 2024
trajs_filenames = ['LSZH_MAY24_1.pkl', 'LSZH_MAY24_2.pkl', 'LSZH_MAY24_3.pkl']
start = ["2024-05-01 00:00","2024-05-11 00:00", "2024-05-21 00:00"]
stop = ["2024-05-10 23:59","2024-05-20 23:59", "2024-05-31 23:59"]

# trajs_filenames = ['LSZH_MAY24_2.pkl', 'LSZH_MAY24_3.pkl']
# start = ["2024-05-11 00:00", "2024-05-21 00:00"]
# stop = ["2024-05-20 23:59", "2024-05-31 23:59"]

# # June 2024
# trajs_filenames = ['LSZH_JUN24_1.pkl', 'LSZH_JUN24_2.pkl', 'LSZH_JUN24_3.pkl']
# start = ["2024-06-01 00:00","2024-06-11 00:00", "2024-06-21 00:00"]
# stop = ["2024-06-10 23:59","2024-06-20 23:59", "2024-06-30 23:59"]

# # July 2024
# trajs_filenames = ['LSZH_JUL24_1.pkl', 'LSZH_JUL24_2.pkl', 'LSZH_JUL24_3.pkl']
# start = ["2024-07-01 00:00","2024-07-11 00:00", "2024-07-21 00:00"]
# stop = ["2024-07-10 23:59","2024-07-20 23:59", "2024-07-31 23:59"]

# # August 2024
# trajs_filenames = ['LSZH_AUG24_1.pkl', 'LSZH_AUG24_2.pkl', 'LSZH_AUG24_3.pkl']
# start = ["2024-08-01 00:00","2024-08-11 00:00", "2024-08-21 00:00"]
# stop = ["2024-08-10 23:59","2024-08-20 23:59", "2024-08-31 23:59"]

# # September 2024
# trajs_filenames = ['LSZH_SEP24_1.pkl', 'LSZH_SEP24_2.pkl', 'LSZH_SEP24_3.pkl']
# start = ["2024-09-01 00:00","2024-09-11 00:00", "2024-09-21 00:00"]
# stop = ["2024-09-10 23:59","2024-09-20 23:59", "2024-09-30 23:59"]


In [ ]:
from agps_funs import alternative_pushback_detection, takeoff_detection, get_df_movements, add_known_missing_icao24, remove_helos_and_outliers
from agps_config import get_Stands_LSZH
from traffic.data import aircraft

# Function to correct format of timestamps
def convert_timestamp(x):
    if isinstance(x, (float, int)) and x > 1e9:  # Convert to unix timestamps if float or int.
        return pd.to_datetime(x, unit='s', errors='coerce')
    elif isinstance(x, pd.Timestamp): # If x is alreardy a pandas timestamp, x will be returned
        return x
    else:
        return pd.NaT  # Invalid Value will be converted to Not a Timestamp

# Specify the available number of CPUs for computation
max_workers=8

i = 0
while i < len(trajs_filenames):
    print(trajs_filenames[i])
    
    # Download data from the OSN
    timestamps = pd.date_range(start[i], stop[i], freq="4h")
    data = []
    for t1, t2 in tqdm(zip(timestamps[:-1], timestamps[1:]), total=len(timestamps) - 1, desc="Processing timestamps"):
            tmp = opensky.history(start=str(t1), stop=str(t2), bounds=airports[airport_str].shape.convex_hull.buffer(0.1))
            if tmp is not None:
                data.append(tmp.data)
            
    gnd_trajs = pd.concat(data)
    
    # Make sure data format is "correct"
    gnd_trajs['timestamp'] = gnd_trajs.data['timestamp'].apply(convert_timestamp)
    gnd_trajs['hour'] = gnd_trajs.data['hour'].apply(convert_timestamp)
    gnd_trajs['onground'] = gnd_trajs.data['onground'].astype(np.bool_)
    gnd_trajs['alert'] = gnd_trajs.data['alert'].astype(np.bool_)
    gnd_trajs['spi'] = gnd_trajs.data['spi'].astype(np.bool_)
    gnd_trajs['last_position'] = gnd_trajs.data['last_position'].astype(np.float64)
    gnd_trajs['icao24'] = gnd_trajs.data['icao24'].astype('object')
    gnd_trajs['callsign'] = gnd_trajs.data['callsign'].astype('object')
    gnd_trajs['squawk'] = gnd_trajs.data['squawk'].astype('object')

    # Retain only points near the ground or on the ground
    gnd_trajs = gnd_trajs.query('altitude<4000 or onground')
    gnd_trajs = Traffic(gnd_trajs)

    # Resample data & add additional columns
    gnd_trajs = gnd_trajs.assign_id().resample('1s').eval(max_workers=max_workers, desc='resampling')
    gnd_trajs = gnd_trajs.aircraft_data()
    gnd_trajs = gnd_trajs.cumulative_distance().eval(max_workers=max_workers, desc='cumdist')

    # Get icaoaircrafttype (if there is any)
    aircraftdata=aircraft.data
    aircraftdata['icao24'] = aircraftdata['icao24'].str.strip().str.lower()
    gnd_trajs = gnd_trajs.merge(aircraftdata[['icao24', 'icaoaircrafttype']], 
                                on='icao24', 
                                how='left')

    # Classify takeoffs
    gnd_trajs = gnd_trajs.iterate_lazy().pipe(takeoff_detection, 'lszh').eval(max_workers=max_workers, desc='Takeoff Detection')

    # Classify pushbacks
    stands = get_Stands_LSZH()
    gnd_trajs = gnd_trajs.iterate_lazy().pipe(alternative_pushback_detection, stands).eval(max_workers=max_workers, desc='Pushback Detection')

    # Save traj data
    gnd_trajs.to_pickle(trajs_filenames[i])

    # Get df_movements & save
    df_movements = get_df_movements(gnd_trajs)

    # Add some (known missing) icao24
    df_movements = add_known_missing_icao24(df_movements)

    # Remove outliers, helos, etc.
    df_movements = remove_helos_and_outliers(df_movements)

    # Save to pickle
    save_filename = trajs_filenames[i].replace('.pkl', '_df_movements.pkl')
    df_movements.to_pickle(save_filename)

    i +=1


***
# Fuel Estimation
***

In [2]:
import agps_funs
from agps_config import AC2CONSIDER
importlib.reload(agps_funs)

# Concatinate df_movements of multiple months
file_paths = ['LSZH_MAY24_1_df_movements.pkl', 'LSZH_MAY24_2_df_movements.pkl', 'LSZH_MAY24_3_df_movements.pkl',
              'LSZH_JUN24_1_df_movements.pkl', 'LSZH_JUN24_2_df_movements.pkl', 'LSZH_JUN24_3_df_movements.pkl',
              'LSZH_JUL24_1_df_movements.pkl', 'LSZH_JUL24_2_df_movements.pkl', 'LSZH_JUL24_3_df_movements.pkl',
              'LSZH_AUG24_1_df_movements.pkl', 'LSZH_AUG24_2_df_movements.pkl', 'LSZH_AUG24_3_df_movements.pkl',
              'LSZH_SEP24_1_df_movements.pkl', 'LSZH_SEP24_2_df_movements.pkl', 'LSZH_SEP24_3_df_movements.pkl',
              ]


df_list = [pd.read_pickle(file) for file in file_paths]
df_movements = pd.concat(df_list, ignore_index=True).convert_dtypes()

df_movements = df_movements.query('isTakeoff')
df_movements = df_movements[df_movements['typecode'].isin(AC2CONSIDER)]

# Get some aircraft-related data
df_movements = agps_funs.getNengine_df(df_movements)
df_movements = agps_funs.getIdleFF_df(df_movements)
df_movements = agps_funs.getAPUfuelFlow_df(df_movements)

# Estimate Fuel Consumption for Normal Taxi & external AGPS
df_movements = agps_funs.normalTaxiFuel_df(df_movements)
df_movements = agps_funs.extAGPSTaxiFuel_df(df_movements)

df_movements = agps_funs.normalTaxiFuel_df(df_movements, 
                                           startupTime=60,
                                           warmupTime=180,
                                           colNames=['MESengine180', 'MESapu180', 'normTAXIengine180']
                                           )

df_movements.to_pickle('LSZH_MAY_SEP_df_movements.pkl')
df_movements

/var/folders/84/0lntlwrj46zgpwppnsr6znzw0000gr/T/ipykernel_93274/681712110.py:17: RuntimeWarning: Engine has switched to 'python' because numexpr does not support extension array dtypes. Please set your engine to python manually.
  df_movements = df_movements.query('isTakeoff')


,flight_id,icao24,callsign,isTakeoff,isPushback,startPushback,startTaxi,lineupTime,taxiDuration,taxiDistance,...,APUhighFF,APUnormalFF,MESengine,MESapu,normTAXIengine,extAGPSapu,extAGPStug,MESengine180,MESapu180,normTAXIengine180
2,AAL93_11938,aae1ea,AAL93,True,False,NaT,2024-05-05 09:48:15+00:00,2024-05-05 10:04:55+00:00,0 days 00:16:40,1.356054,...,262,238,93.66,16.666667,446.000,66.111111,NaN,120.42,20.633333,446.000
3,AAL93_11941,aae597,AAL93,True,False,NaT,2024-05-01 09:51:09+00:00,2024-05-01 10:03:06+00:00,0 days 00:11:57,1.354637,...,262,238,93.66,16.666667,319.782,47.401667,NaN,120.42,20.633333,319.782
4,AAL93_11945,ab1839,AAL93,True,False,NaT,2024-05-09 10:01:50+00:00,2024-05-09 10:19:47+00:00,0 days 00:17:57,1.112419,...,262,238,93.66,16.666667,480.342,71.201667,NaN,120.42,20.633333,480.342
5,AAL93_11947,ab1fa7,AAL93,True,True,2024-05-03 09:43:46+00:00,2024-05-03 09:44:57+00:00,2024-05-03 09:56:52+00:00,0 days 00:11:55,0.426034,...,262,238,93.66,16.666667,318.890,47.269444,NaN,120.42,20.633333,318.890
6,AAL93_11950,ab236a,AAL93,True,False,NaT,2024-05-04 09:41:23+00:00,2024-05-04 09:52:56+00:00,0 days 00:11:33,0.788887,...,262,238,93.66,16.666667,309.078,45.815000,NaN,120.42,20.633333,309.078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57567,VLG7516_174,344099,VLG7516,True,False,NaT,2024-09-27 12:52:06+00:00,2024-09-27 12:55:13+00:00,0 days 00:03:07,0.322391,...,130,110,44.94,8.000000,40.018,5.713889,NaN,57.78,9.833333,40.018
57568,VLG7516_222,344697,VLG7516,True,False,NaT,2024-09-30 08:53:26+00:00,2024-09-30 08:55:29+00:00,0 days 00:02:03,0.386516,...,130,110,44.94,8.000000,26.322,3.758333,NaN,57.78,9.833333,26.322
57569,VLG7516_247,345645,VLG7516,True,False,NaT,2024-09-23 08:40:30+00:00,2024-09-23 08:46:04+00:00,0 days 00:05:34,0.922098,...,130,110,44.94,8.000000,71.476,10.205556,NaN,57.78,9.833333,71.476
57570,VLG7516_270,346090,VLG7516,True,False,NaT,2024-09-25 12:47:27+00:00,2024-09-25 12:52:50+00:00,0 days 00:05:23,0.34366,...,130,110,33.60,8.000000,51.680,9.869444,NaN,43.20,9.833333,51.680
